In [1]:
import fasttext
import pandas as pd 
import numpy as np 
import gc 
import os 
import nltk

/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/chen/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
from collections import Counter

from tqdm.autonotebook import tqdm

/home/chen/anaconda3/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


# 读取数据并进行预处理

In [2]:
data = pd.read_csv("../data/review.csv")

In [3]:
data.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,Q1sbwvVQXV2734tPgoKj4Q,hG7b0MtEbXx5QzbzE6C_VA,ujmEBvifdJM6h6RLv4wQIg,1.0,6,1,0,Total bill for this horrible service? Over $8G...,2013-05-07 04:34:36
1,GJXCdrto3ASJOqKeVWPi6Q,yXQM5uF2jS6es16SJzNHfg,NZnhc2sEQy3RmzKTZnqtwQ,5.0,0,0,0,I *adore* Travis at the Hard Rock's new Kelly ...,2017-01-14 21:30:33
2,2TzJjDVDEuAW6MR5Vuc1ug,n6-Gk65cPZL6Uz8qRm3NYw,WTqjgwHlXbSFevF32_DJVw,5.0,3,0,0,I have to say that this office really has it t...,2016-11-09 20:09:03
3,yi0R0Ugj_xUx_Nek0-_Qig,dacAIZ6fTM6mqwW5uxkskg,ikCg8xy5JIg_NGPx-MSIDA,5.0,0,0,0,Went in for a lunch. Steak sandwich was delici...,2018-01-09 20:56:38
4,11a8sVPMUFtaC7_ABRkmtw,ssoyf2_x0EQMed6fgHeMyQ,b1b1eb3uo-w561D0ZfCEiQ,1.0,7,0,0,Today was my second out of three sessions I ha...,2018-01-30 23:07:38


In [4]:
# 删除掉无用的列，只保留有用的列
data = data[['review_id', 'stars', 'text']]

In [6]:
# 对标签进行统计分析
Counter(data['stars'])

Counter({1.0: 352637, 5.0: 1049906, 4.0: 525826, 3.0: 263437, 2.0: 191981})

In [7]:
# 对标签进行处理，大小3的表示pos，小于等于3的表示neg
data['emotion'] = data['stars'].apply(lambda x: int(x>3))

In [8]:
data.head()

,review_id,stars,text,emotion
0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,0
1,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,1
2,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,1
3,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,1
4,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,0


# 对文本进行处理

In [33]:
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
import spacy
import re 
from nltk.stem import WordNetLemmatizer

stoplists = stopwords.words("english")

wnl = WordNetLemmatizer()

In [28]:
# 添加需要去除的标点符号集，问号和感叹号除外
puncts = [',', '.', '"', ':', ')', '(', '-', '?', '!', '|', ';', "'", '$', '&', '/', 
          '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', '·', '_', 
          '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', 
          '§', '″', '′', 'Â', '█', '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', 
          '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥',
          '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', 
          '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
          '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', 
          '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

# 定义一些常见的缩写
contraction_patterns = [ (r'won\'t', 'will not'), (r'can\'t', 'cannot'), (r'i\'m', 'i am'),
                        (r'isn\'t', 'is not'), (r'(\w+)\'ll', '\g<1> will'), 
                        (r'(\w+)n\'t', '\g<1> not'),(r'(\w+)\'ve', '\g<1> have'), 
                        (r'(\w+)\'s', '\g<1> is'), (r'(\w+)\'re', '\g<1> are'), 
                        (r'(\w+)\'d', '\g<1> would'), (r'&', 'and'), (r'dammit', 'damn it'),
                        (r'dont', 'do not'), (r'wont', 'will not') ]

In [34]:
def clean_text(text):
    # 去除对情感分类没有用的数字
    text = re.sub("[0-9]+", "", text)
    # 对重复出现的标点进行天魂
    text = re.sub(r"(\!)\1+", "multiExclamation", text)
    text = re.sub(r"(\?)\1+", "multiQuestion", text)
    text = re.sub(r"(\.)\1+", "multiStop", text)
    
    # 对缩写进行替换
    patterns = [(re.compile(regex), repl) for (regex, repl) in contraction_patterns]
    for (pattern, repl) in patterns:
        (text, count) = re.subn(pattern, repl, text)
    
    # 在标点前面加上空格
    for punct in puncts:
        text = text.replace(punct, f" {punct} ")
    
    #print(text)
    # 对文本进行分词
    text_split = tokenize.word_tokenize(text)
    text = [word for word in text_split if word not in stoplists]
    text = [wnl.lemmatize(word) for word in text]
    
    return " ".join(text)

In [35]:
clean_text("I'll a test file ! ... !@@##$()")

'I test file ! multiStop ! @ @ # # $ ( )'

In [36]:
data['text'] = data['text'].apply(lambda s: clean_text(s))

In [37]:
data.head()

,review_id,stars,text,emotion
0,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill horrible service ? Over $ Gs . Thes...,0
1,GJXCdrto3ASJOqKeVWPi6Q,5.0,I * adore * Travis Hard Rock new Kelly Cardena...,1
2,2TzJjDVDEuAW6MR5Vuc1ug,5.0,"I say office really together , organized frien...",1
3,yi0R0Ugj_xUx_Nek0-_Qig,5.0,"Went lunch . Steak sandwich delicious , Caesar...",1
4,11a8sVPMUFtaC7_ABRkmtw,1.0,Today second three session I paid . Although f...,0


In [38]:
# 划分训练集和验证集
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(data['text'].values, data['emotion'].values,
                                                   test_size=0.3, random_state=2019, shuffle=True,
                                                   stratify=data['emotion'].values)

In [40]:
X_train[0]

'Big Juicy Gourmet Burger melted cheese bacon crumbles underneath . salted golden shoestring fry compliment juicy thick patty beef well . The beef ground site enjoyed medium rare burger . Very delicious . On bar `` Whisky Book `` , filled diffferent whisky . If like Bourbon Whisky enough reason go . Wish beer , feeling scratched surface . burger $ , beer $ hh .'

下面将文本写成fasttext需要使用的文本格式，比如：

Big Juicy Gourmet Burger melted cheese bacon crumbles underneath . &emsp;&emsp;   \_\_label\_\_pos 

即每一行前面表示文本，后面表示标签

In [42]:
# 将训练集和测试集写入文件中
with open("../data/yelps_train_fasttext.txt", "w", encoding="utf8") as f: 
    train = zip(X_train, y_train)
    for X, y in tqdm(train):
        label = "pos" if y > 0 else "neg"
        record = X + "\t__label__"+label+"\n"
        f.write(record)

In [43]:
# 将测试集写入文件中
with open("../data/yelps_test_fasttext.txt", "w", encoding="utf-8") as f: 
    test = zip(X_test, y_test)
    for X, y in tqdm(test):
        label = "pos" if y>0 else "neg"
        record = X+"\t__label__"+label+"\n"
        f.write(record)

In [56]:
X_test.shape

(715137,)

# 训练fastText模型

## 对yelps数据集进行分类

In [46]:
classifier = fasttext.train_supervised("../data/yelps_train_fasttext.txt", epoch=25, lr=0.25, wordNgrams=2,
                                      verbose=2, minCount=2)

In [51]:
# 测试训练的结果
result = classifier.test("../data/yelps_test_fasttext.txt")

In [54]:
# 第一个参数表示测试集大小，第二个参数表示准确率，第三个参数表示召回率
result

(715137, 0.9033835474881037, 0.9033835474881037)

- 共测试了715137条评论

- <font color=red size=4>**得到 P为90.338%，R为90.338%**</font>

In [55]:
# 预测结果第一个参数表示标签，第二个参数表示概率
classifier.predict(["This is bad !"])

([['__label__neg']], array([[0.96849835]]))

## 对IMDB数据集进行分类

In [3]:
classifier = fasttext.train_supervised("../data/preprocess/imdb_fasttext_train.txt", epoch=25, lr=0.25,
                                      wordNgrams=2, verbose=2, minCount=2)

In [4]:
# 测试集结果
result = classifier.test("../data/preprocess/imdb_fasttext_test.txt")

In [6]:
result

(6250, 0.888, 0.888)

- 共测试了6250条评论

- <font color=red size=4>**得到P为88.8%，R为88.8%**</font>

# 用tensorflow实现fastText

In [58]:
import tensorflow as tf
import numpy as np 
import os 

In [59]:
class fastTextModel():
    def __init__(self, sequence_length, num_classes, vocab_size, embedding_size, 
                learning_rate, decay_steps, decay_rate, l2_reg_lambda, is_training=True, 
                initializer=tf.random_normal_initializer(stddev=0.1)):
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_classes = num_classes
        self.sequence_length = sequence_length
        self.learning_rate = learning_rate
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.is_training = is_training
        self.l2_reg_lambda = l2_reg_lambda
        self.initializer = initializer
        
        self.input_x = tf.placeholder(tf.int32, [None, self.sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.int32, [None, self.num_classes], name="input_y")
        
        self.global_step = tf.Variable(0, trainable=False, name="global_step")
        self.instantiate_weight()
        self.logits = self.inference()
        self.loss_val = self.loss()
        self.train_op = self.train() 
        
        self.predictions = tf.argmax(self.logits, axis=1, name="predictions")
        correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32), name="accuracy")
        
    def instantiate_weight(self):
        with tf.name_scope("weights"):
            self.Embedding = tf.get_variable("Embedding", shape=[self.vocab_size, self.embedding_size],
                                            initializer=self.initializer)
            self.W_projection = tf.get_variable("W_projection", shape=[self.embedding_size, self.num_classes],
                                               initializer=self.initializer)
            self.b_projection = tf.get_variable("b_projection", shape=[self.num_classes])
            
    def inference(self):
        '''
        1. word embedding
        2. average embedding
        3. linear classifier
        '''
        # embedding layer
        with tf.name_scope("embedding"):
            words_embedding = tf.nn.embedding_lookup(self.Embedding, self.input_x)
            self.average_embedding = tf.reduce_mean(words_embedding, axis=1)  # 代表了这句话的embedding
        
        logits = tf.matmul(self.average_embedding, self.W_projection) + self.b_projection
        return logits
    
    def loss(self):
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.input_y, logits=self.logits)
            data_loss = tf.reduce_mean(losses)
            l2_loss = tf.add_n([tf.nn.l2_loss(cand_var)] for cand_var in tf.trainable_variables() 
                              if "bias" not in cand_var.name) * self.l2_reg_lambda
            data_loss += l2_loss
            return data_loss
        
    def train(self):
        with tf.name_scope("train"):
            learning_rate = tf.train.exponential_decay(self.learning_rate, self.global_step, 
                                                      self.decay_steps, self.decay_rate,
                                                      staircase=True)
            train_op = tf.contrib.layers.optimize_loss(self.loss_val, 
                                                      global_step=self.global_step,
                                                      learning_rate=learning_rate,
                                                      optimizer="Adam")
            
            return train_op